In [1]:
import os
from glob import glob

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import mne
mne.set_log_level(verbose=False)

from Datasets import *

In [2]:
BATCH_SIZE = 4
channels = ['F4']

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
%%time
datasets = [EDFData_PTH(path, channels=channels) for path in glob("../Data/*edf")]
dataset_together = torch.utils.data.ConcatDataset(datasets)
dataloaders = [torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, drop_last=True) for dataset in datasets]
dataloader_together = torch.utils.data.DataLoader(dataset_together, batch_size = BATCH_SIZE, drop_last=True)
len(datasets), len(dataloaders), len(dataset_together), len(dataloader_together)

e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Omitted 3 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Omitted 109 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Omitted 92 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Omitted 120 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: R

(14, 14, 11157, 2789)

In [7]:
[print(dataset.epochs) for dataset in datasets]
print()

<Epochs |  882 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 25
 'Sleep stage N2': 317
 'Sleep stage N3': 193
 'Sleep stage R': 101
 'Sleep stage W': 246>
<Epochs |  756 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 14
 'Sleep stage N2': 455
 'Sleep stage N3': 114
 'Sleep stage W': 173>
<Epochs |  715 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 4
 'Sleep stage N2': 248
 'Sleep stage N3': 185
 'Sleep stage R': 93
 'Sleep stage W': 185>
<Epochs |  764 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 14
 'Sleep stage N2': 403
 'Sleep stage N3': 218
 'Sleep stage R': 94
 'Sleep stage W': 35>
<Epochs |  805 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 127
 'Sleep stage N2': 293
 'Sleep stage N3': 169
 'Sleep stage R': 86
 'Sleep stage W': 130>
<Epochs |  780 events

In [8]:
sampling_rate = int(datasets[0].sampling_rate)
sampling_rate

512

In [58]:
class TinySleep(nn.Module):
    def __init__(self, sampling_rate, channels):
        super(TinySleep, self).__init__()
        self.sampling_rate = sampling_rate
        self.channels = channels

        self.feature_extraction = nn.Sequential(*[
            nn.Conv1d(in_channels=len(channels), out_channels=128, kernel_size=sampling_rate//2, stride=sampling_rate//4),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=8, stride=8),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=8, stride=1, padding=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=8, stride=1, padding=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=8, stride=1, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        ])

        self.lstm = nn.LSTM(input_size=128, hidden_size=128, batch_first=True)
        
        self.classifier = nn.Linear(2*128, 10)

    def forward(self, X):
        X = self.feature_extraction(X)
        X, _ = self.lstm(X.permute(0,2,1))
        X = self.classifier(X.reshape(X.shape[0],-1))
        return X

In [72]:
model = TinySleep(sampling_rate, channels)
model.to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
model

TinySleep(
  (feature_extraction): Sequential(
    (0): Conv1d(1, 128, kernel_size=(256,), stride=(128,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(3,))
    (4): ReLU()
    (5): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(3,))
    (6): ReLU()
    (7): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(3,))
    (8): ReLU()
    (9): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(128, 128, batch_first=True)
  (classifier): Linear(in_features=256, out_features=10, bias=True)
)

In [63]:
with torch.no_grad():
    pred = model(a)
pred.shape

torch.Size([4, 10])

In [74]:
%%time
EPOCHS = 1

for epoch in range(EPOCHS):
    for X, Y in dataloader_together:
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, torch.squeeze(Y).long())
        loss.backward()
        optimizer.step()
    break

Wall time: 34 ms


In [68]:
pred.shape, Y.shape

(torch.Size([4, 10]), torch.Size([4, 1]))